In [1]:
from parse_data import get_data, get_modified_values
from rnn import PredictTime
import torch

df = get_data()

In [2]:

def make_data(df):

    x_train, y_train = [], []
    prev = None

    for row in df:
        if prev is None:
            prev = row
        x_train.append(prev)
        y_train.append(row)
        prev = row
    return torch.stack(x_train).float(),torch.stack(y_train).float()

make_data(get_modified_values(get_data()))



(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]]))

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from IPython.display import clear_output
import torch.nn as nn

batch_size = 10
x_d, y_d = make_data(get_modified_values(get_data()))

model = PredictTime(input_size=x_d[0].size()[0],
                    output_size=y_d[0].size()[0],
                    hidden_layers=10,
                    hidden_size=100)
optimizer = optim.Adam(model.parameters(),lr=0.01)

loss = nn.CrossEntropyLoss()
loader = data.DataLoader(data.TensorDataset(x_d,y_d), batch_size=batch_size)
epochs = 1000
for e in range(epochs):
    model.train()
    #print(next(iter(loader)))
    model.clean_state()
    for x, y in loader:
                
        model.init_state()
        y_pred = model(x)
        l = loss(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    if e % 10 != 0:
        continue
    #clear_output(wait=True)
    sum_loss = 0

    for x, y in loader:
        model.eval()
        with torch.no_grad():
            y_pred = model(x)
            sum_loss += np.sqrt(loss(y_pred, y))
    
    print("Epoch %d Loss %.4f" % (e, sum_loss))
    
    
#for d in df.values:

Epoch 0 Loss 1153.3618
Epoch 10 Loss 1145.5801
Epoch 20 Loss 1144.0941
Epoch 30 Loss 1142.9624
Epoch 40 Loss 1143.9106
Epoch 50 Loss 1142.5227
